## 13. 텐서플로에서 데이터 적재와 전처리하기

## 패키지 import

In [107]:
import sys
import sklearn

import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import tarfile
import urllib.request
import pandas as pd

In [108]:
import tensorflow as tf
from tensorflow import keras

## 필요 데이터셋

In [4]:
X = tf.range(10)
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

## 13.2 TFRecord 포맷
* 이진포맷 
*  tf.io.TFRecordWriter 클래스 사용해서 TFRecord 생성

In [5]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

* tf.data.TFRecordDataset 사용하여 TFRecord 읽기

In [6]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


### 13.2.1 압축된 TFRecord 파일
* options 매개변수를 사용해 TFRecord 파일 압축 가능함
* 압축형식을 지정하여 압축된 TFRecord 파일 읽기

In [7]:
options = tf.io.TFRecordOptions(compression_type="GZIP")
with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f:
    f.write(b"This is the first record")
    f.write(b"And this is the second record")

In [8]:
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")
for item in dataset:
    print(item)

tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)


### 13.2.2 프로토콜 버퍼 개요
* 자체적인 프로토콜 버퍼를 생성
* 1) 프로토콜 버퍼 생성 시 컴파일러(protoc)를 설치하고 
* https://developers.google.com/protocol-buffers/docs/downloads
* 2) 프로토콜 버퍼 정의를 컴파일하여 코드에서 사용할 수 있는 파이썬 모듈 만들기

In [9]:
%%writefile person.proto
syntax = "proto3";
message Person {
  string name = 1;
  int32 id = 2;
  repeated string email = 3;
}

Overwriting person.proto


In [10]:
!protoc person.proto --python_out=. --descriptor_set_out=person.desc --include_imports

In [11]:
!dir person*

 C 드라이브의 볼륨에는 이름이 없습니다.
 볼륨 일련 번호: C077-EFC1

 C:\Users\iyr0219\Documents\핸즈온 머신러닝 디렉터리

2021-08-15  오후 07:42                92 person.desc
2021-08-15  오후 07:42               106 person.proto
2021-08-15  오후 07:42             2,732 person_pb2.py
               3개 파일               2,930 바이트
               0개 디렉터리  122,014,093,312 바이트 남음


In [12]:
from person_pb2 import Person # 생성된 클래스 import

In [13]:
person = Person(name="Al", id=123, email=["a@b.com"])  # Person 객체 생성
print(person)  # Person 출력

name: "Al"
id: 123
email: "a@b.com"



In [14]:
person.name  # 필드 읽기

'Al'

In [15]:
person.name = "Alice"  # 필드 수정

In [16]:
person.email[0]  # 반복필드는 배열처럼 참조 가능

'a@b.com'

In [17]:
person.email.append("c@d.com")  # 이메일 주소 추가

In [18]:
s = person.SerializeToString()  # 바이트 문자열로 객체 직렬화
s

b'\n\x05Alice\x10{\x1a\x07a@b.com\x1a\x07c@d.com'

In [19]:
person2 = Person()  # 새로운 Person 생성
person2.ParseFromString(s)  # 바이트 문자열 파싱 (27 바이트)

27

In [20]:
person == person2  # 동일

True

### 13.2.3 텐서플로 프로토콜 버퍼
* 텐서플로는 파싱연산을 제공하기 위한 특별한 프로토콜 버퍼 정의를 가지고 있음

In [25]:
# Example 프로토콜버퍼: 데이터셋에 있는 하나의 샘플을 표현
# Example 프로토콜 버퍼 정의
# Feature는 BytesList,FloatList, Int64List 중 하나 가지고 있음
# Features는 특성 이름과 특성값을 매핑한 딕셔너리 가짐

In [24]:
%%writefile person.proto
syntax = "proto3";
message BytesList { repeated bytes value = 1; }
message FloatList { repeated float value = 1 [packed = true]; }
message Int64List { repeated int64 value = 1 [packed = true]; }
message Feature {
    oneof kind {
        BytesList bytes_list = 1;
        FloatList float_list = 2;
        Int64List int64_list = 3;
    }
};
message Features { map<string, Feature> feature = 1; };
message Example { Features features = 1; };

Overwriting person.proto


In [28]:
# tf.train.Example 객체를 만들고 TFRecord 파일에 저장
import tensorflow as tf
BytesList = tf.train.BytesList
FloatList = tf.train.FloatList
Int64List = tf.train.Int64List
Feature = tf.train.Feature
Features = tf.train.Features
Example = tf.train.Example

# Example 프로토콜 버퍼 생성
person_example = Example(
    features=Features(
        feature={
            "name": Feature(bytes_list=BytesList(value=[b"Alice"])),
            "id": Feature(int64_list=Int64List(value=[123])),
            "emails": Feature(bytes_list=BytesList(value=[b"a@b.com", b"c@d.com"]))
        }))

# SerializeToString 호출하려 직렬화
# 결과 데이터 TFRcord 로 저장
with tf.io.TFRecordWriter("my_contacts.tfrecord") as f:
    f.write(person_example.SerializeToString())

### 13.2.4 Example 프로토콜 버퍼를 읽고 파싱하기
* 각 Example 파싱

In [29]:

# 설명 딕셔너리를 정의
feature_description = {
    "name": tf.io.FixedLenFeature([], tf.string, default_value=""),
    "id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "emails": tf.io.VarLenFeature(tf.string),
}
#TFRecordDataset 돌면서 데이터셋에 포함된 직렬화된 Example 프로토콜 버퍼 파싱
for serialized_example in tf.data.TFRecordDataset(["my_contacts.tfrecord"]):
    parsed_example = tf.io.parse_single_example(serialized_example, # 직렬화된 데이터를 담은 문자열 스칼라 텐서
                                                feature_description) # 각 특성에 대한 설명

In [30]:
parsed_example

{'emails': <tensorflow.python.framework.sparse_tensor.SparseTensor at 0x1c0a1048940>,
 'id': <tf.Tensor: id=71, shape=(), dtype=int64, numpy=123>,
 'name': <tf.Tensor: id=72, shape=(), dtype=string, numpy=b'Alice'>}

In [31]:
tf.sparse.to_dense(parsed_example["emails"], default_value=b"")

<tf.Tensor: id=74, shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [32]:
parsed_example["emails"].values

<tf.Tensor: id=69, shape=(2,), dtype=string, numpy=array([b'a@b.com', b'c@d.com'], dtype=object)>

In [33]:
# tf.io.parse_example로 배치 단위로 파싱 가능
dataset = tf.data.TFRecordDataset(["my_contacts.tfrecord"]).batch(10)
for serialized_examples in dataset:
    parsed_examples = tf.io.parse_example(serialized_examples,
                                          feature_description)

### 13.2.5 Sequence Example 프로토콜 버퍼를 읽고 파싱하기
* 리스트의 리스트 다룰때 유용

In [36]:
# sequence example 프로토콜 정의
# 하나의 features 객체와 이름이 있는 한 개 이상의 FeatureList를 가진 FeatureLists 객체를 포함

In [35]:
%%writefile person.proto
syntax = "proto3";

message FeatureList { repeated Feature feature = 1; };
message FeatureLists { map<string, FeatureList> feature_list = 1; };
message SequenceExample {
  Features context = 1;
  FeatureLists feature_lists = 2;
};

Overwriting person.proto


In [37]:
FeatureList = tf.train.FeatureList
FeatureLists = tf.train.FeatureLists
SequenceExample = tf.train.SequenceExample

context = Features(feature={
    "author_id": Feature(int64_list=Int64List(value=[123])),
    "title": Feature(bytes_list=BytesList(value=[b"A", b"desert", b"place", b"."])),
    "pub_date": Feature(int64_list=Int64List(value=[1623, 12, 25]))
})

content = [["When", "shall", "we", "three", "meet", "again", "?"],
           ["In", "thunder", ",", "lightning", ",", "or", "in", "rain", "?"]]
comments = [["When", "the", "hurlyburly", "'s", "done", "."],
            ["When", "the", "battle", "'s", "lost", "and", "won", "."]]

def words_to_feature(words):
    return Feature(bytes_list=BytesList(value=[word.encode("utf-8")
                                               for word in words]))

content_features = [words_to_feature(sentence) for sentence in content]
comments_features = [words_to_feature(comment) for comment in comments]
 
# Sequence Example 프로토콜 버퍼 생성
sequence_example = SequenceExample(
    context=context,
    feature_lists=FeatureLists(feature_list={
        "content": FeatureList(feature=content_features),
        "comments": FeatureList(feature=comments_features)
    }))

In [38]:
# 문맥특성과 특성 리스트를 담은 튜플을 반환
serialized_sequence_example = sequence_example.SerializeToString()

In [40]:
# 설명 딕셔너리를 정의
context_feature_descriptions = {
    "author_id": tf.io.FixedLenFeature([], tf.int64, default_value=0),
    "title": tf.io.VarLenFeature(tf.string),
    "pub_date": tf.io.FixedLenFeature([3], tf.int64, default_value=[0, 0, 0]),
}
sequence_feature_descriptions = {
    "content": tf.io.VarLenFeature(tf.string),
    "comments": tf.io.VarLenFeature(tf.string),
}

In [42]:
# 직렬화된Sequence Example 프로토콜 버퍼 파싱
parsed_context, parsed_feature_lists = tf.io.parse_single_sequence_example(
    serialized_sequence_example, context_feature_descriptions,
    sequence_feature_descriptions)

## 13.3 입력 특성 전처리
### 13.3.1 원핫벡터를 사용해 범주형 특성 인코딩하기
* 범주형 변수를 숫자로 전처리

In [57]:
housing = pd.read_csv('housing.csv')

In [59]:
housing.head(3)

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,NEAR BAY
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,NEAR BAY
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,NEAR BAY


* ocean proximity 범주형 특성을 원핫인코딩

In [66]:
# 룩업 데이블을 사용하여 각 범주를 인덱스로 매핑
# 어휘사전 정의
ocean_prox_vocab = ['<1H OCEAN', 'INLAND', 'ISLAND', 'NEAR BAY', 'NEAR OCEAN']
# 인덱스의 텐서 생성
indices = tf.range(len(ocean_prox_vocab), dtype=tf.int64)
# lookup 테이블을 위한 초기화 객체 생성
table_init=tf.lookup.KeyValueTensorInitializer(ocean_prox_vocab,indices)
# 초기화 객체와 oov버킷을 지정하여 룩업테이블을 생성
# 어휘사전에 없는 범주의 경우 oov로 할당
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)


In [67]:
# 원핫인코딩
categories = tf.constant(['NEAR BAY', 'DESERT', 'INLAND', 'INLAND'])
cat_indices = table.lookup(categories)

In [68]:
cat_indices

<tf.Tensor: id=186, shape=(4,), dtype=int64, numpy=array([3, 5, 1, 1], dtype=int64)>

In [71]:
cat_one_hot = tf.one_hot(cat_indices, depth=len(ocean_prox_vocab) +num_oov_buckets)

In [72]:
cat_one_hot

<tf.Tensor: id=190, shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0.]], dtype=float32)>

### 13.3.2 인베딩을 사용해 범주형 특성 인코딩하기

In [74]:
# 임베딩 행렬을 만들어 랜덤하게 초기화
embedding_dim = 2 # 임베딩 차원
embed_init = tf.random.uniform([len(ocean_prox_vocab) +num_oov_buckets, embedding_dim])
embedding_matrix = tf.Variable(embed_init)


In [76]:
embedding_matrix

<tf.Variable 'Variable:0' shape=(7, 2) dtype=float32, numpy=
array([[0.44701278, 0.3415135 ],
       [0.50409424, 0.88271344],
       [0.73753977, 0.98607314],
       [0.772148  , 0.32938397],
       [0.85544086, 0.8253056 ],
       [0.00989878, 0.3759582 ],
       [0.95580375, 0.20894802]], dtype=float32)>

In [78]:
# 인코딩
print(cat_indices)
# 임베딩 행렬에서 주어진 인덱스에 해당하는 행 찾음
tf.nn.embedding_lookup(embedding_matrix, cat_indices)

tf.Tensor([3 5 1 1], shape=(4,), dtype=int64)


<tf.Tensor: id=205, shape=(4, 2), dtype=float32, numpy=
array([[0.772148  , 0.32938397],
       [0.00989878, 0.3759582 ],
       [0.50409424, 0.88271344],
       [0.50409424, 0.88271344]], dtype=float32)>

In [85]:
# 임베딩 학습하는 케라스 모델
regular_inputs=keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
# 범주의 인덱스를 찾은 다음 임베딩에서 이 인덱스 찾임
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)

cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
# 인베딩과 일반입력을 연결 ->. 신경망에 주입할 입력 생성
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model=keras.models.Model(inputs=[regular_inputs, categories], outputs=[outputs])

## 13.4 TF변환
* 두 개의 특성을 전처리하는 함수

In [103]:
import tensorflow_transform as tft

def preprocess(inputs):  # 입력특성의 배치
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    standardized_age = tft.scale_to_z_score(median_age - tft.mean(median_age))
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)
    return {
        "standardized_median_age": standardized_age,
        "ocean_proximity_id": ocean_proximity_id}


## 13.5 텐서플로 데이터셋(TFDS) 프로젝트
* 표준 데이터셋 다운

In [109]:
import tensorflow_datasets as tfds

datasets = tfds.load(name="mnist")
mnist_train, mnist_test = datasets["train"], datasets["test"]